<a href="https://colab.research.google.com/github/Fu-kit/WeatherWise_CSC101/blob/main/Weather_Advisor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# First, install required packages
!pip install pyinputplus requests openai

import os
import pyinputplus as pyip
import requests
import json
import re
from datetime import datetime, timedelta

class WeatherWise:
    def __init__(self):
        # Use a free weather API (WeatherAPI.com)
        self.weather_api_key = "YOUR_API_KEY_HERE"  # Get free key from weatherapi.com
        self.base_url = "http://api.weatherapi.com/v1"
        self.current_location = None

    def get_weather_data(self, location, days=2):
        """
        Fetch weather data using a reliable weather API
        Since fetch-my-weather might not exist, we'll use a real API
        """
        try:
            # Using WeatherAPI.com (free tier available)
            url = f"{self.base_url}/forecast.json"
            params = {
                'key': self.weather_api_key,
                'q': location,
                'days': min(days, 3),  # Free tier usually limits to 3 days
                'aqi': 'no',
                'alerts': 'no'
            }

            response = requests.get(url, params=params)

            if response.status_code == 200:
                data = response.json()
                return {
                    'status': 'success',
                    'location': data['location'],
                    'current': data['current'],
                    'forecast': data['forecast']['forecastday']
                }
            else:
                # Try with a mock/demo API key for testing
                return self.get_mock_weather_data(location, days)

        except Exception as e:
            print(f"API Error: {e}")
            return self.get_mock_weather_data(location, days)

    def get_mock_weather_data(self, location, days):
        """
        Fallback mock data for testing when API isn't available
        """
        import random

        mock_data = {
            'status': 'success',
            'location': {'name': location, 'country': 'Demo'},
            'current': {
                'temp_c': random.randint(15, 25),
                'condition': {'text': random.choice(['Sunny', 'Partly cloudy', 'Cloudy'])},
                'humidity': random.randint(40, 80),
                'wind_kph': random.randint(5, 20),
                'precip_mm': random.choice([0, 0, 0, 2, 5])
            },
            'forecast': []
        }

        for i in range(days):
            day_data = {
                'date': (datetime.now() + timedelta(days=i)).strftime('%Y-%m-%d'),
                'day': {
                    'maxtemp_c': random.randint(20, 30),
                    'mintemp_c': random.randint(10, 20),
                    'condition': {'text': random.choice(['Sunny', 'Rainy', 'Cloudy'])},
                    'daily_chance_of_rain': random.randint(0, 80),
                    'totalprecip_mm': random.choice([0, 0, 1, 3, 8])
                }
            }
            mock_data['forecast'].append(day_data)

        print(f"ℹ️  Using demo weather data for {location}")
        return mock_data

    def parse_weather_question(self, question):
        """
        Parse natural language questions with fallback logic
        Since hands-on-ai might not exist, we'll use keyword-based parsing
        """
        question_lower = question.lower()

        # Extract intent
        intent = "general"
        if any(word in question_lower for word in ['umbrella', 'rain', 'wet', 'precipitation']):
            intent = "rain"
        elif any(word in question_lower for word in ['cold', 'hot', 'warm', 'temperature', 'wear', 'dress']):
            intent = "temperature"
        elif any(word in question_lower for word in ['picnic', 'outdoor', 'walk', 'hike', 'activity']):
            intent = "activity"
        elif any(word in question_lower for word in ['jacket', 'coat', 'clothes', 'clothing']):
            intent = "clothing"

        # Extract timeframe
        timeframe = "today"
        if 'tomorrow' in question_lower:
            timeframe = "tomorrow"
        elif any(word in question_lower for word in ['weekend', 'saturday', 'sunday']):
            timeframe = "weekend"
        elif 'week' in question_lower:
            timeframe = "week"

        # Extract location (simple regex)
        location = self.current_location
        location_match = re.search(r'in ([A-Za-z\s]+)', question, re.IGNORECASE)
        if location_match:
            location = location_match.group(1).strip()

        return {
            "intent": intent,
            "location": location,
            "timeframe": timeframe,
            "original_question": question
        }

    def generate_weather_response(self, parsed_question, weather_data):
        """Generate helpful, practical responses"""
        if weather_data['status'] != 'success':
            return "Sorry, I couldn't get weather information for that location."

        intent = parsed_question["intent"]
        timeframe = parsed_question["timeframe"]
        location_name = weather_data['location']['name']

        # Get relevant weather data based on timeframe
        if timeframe == "today":
            weather_info = weather_data['current']
            forecast_day = weather_data['forecast'][0] if weather_data['forecast'] else None
        elif timeframe == "tomorrow" and len(weather_data['forecast']) > 1:
            forecast_day = weather_data['forecast'][1]
            weather_info = forecast_day['day']
        else:
            forecast_day = weather_data['forecast'][0] if weather_data['forecast'] else None
            weather_info = forecast_day['day'] if forecast_day else weather_data['current']

        # Generate response based on intent
        if intent == "rain":
            return self.generate_rain_advice(weather_info, forecast_day, location_name, timeframe)
        elif intent == "temperature" or intent == "clothing":
            return self.generate_clothing_advice(weather_info, forecast_day, location_name, timeframe)
        elif intent == "activity":
            return self.generate_activity_advice(weather_info, forecast_day, location_name, timeframe)
        else:
            return self.generate_general_response(weather_info, forecast_day, location_name, timeframe)

    def generate_rain_advice(self, weather_info, forecast_day, location, timeframe):
        """Generate umbrella/rain advice"""
        if forecast_day:
            rain_chance = forecast_day['day'].get('daily_chance_of_rain', 0)
            rain_mm = forecast_day['day'].get('totalprecip_mm', 0)
        else:
            rain_chance = 20  # Default for current weather
            rain_mm = weather_info.get('precip_mm', 0)

        if rain_chance > 70 or rain_mm > 5:
            return f"🌧️ Yes, definitely bring an umbrella! {rain_chance}% chance of rain {timeframe} in {location}."
        elif rain_chance > 30:
            return f"☔ You might want to bring an umbrella just in case. {rain_chance}% chance of rain {timeframe} in {location}."
        else:
            return f"☀️ You probably don't need an umbrella {timeframe}. Only {rain_chance}% chance of rain in {location}."

    def generate_clothing_advice(self, weather_info, forecast_day, location, timeframe):
        """Generate clothing advice based on temperature"""
        if forecast_day:
            max_temp = forecast_day['day'].get('maxtemp_c', 20)
            min_temp = forecast_day['day'].get('mintemp_c', 15)
        else:
            max_temp = min_temp = weather_info.get('temp_c', 20)

        if max_temp < 5:
            advice = "heavy winter coat and warm layers"
        elif max_temp < 15:
            advice = "jacket or sweater"
        elif max_temp < 25:
            advice = "light jacket or long sleeves"
        else:
            advice = "light clothing like t-shirts"

        return f"🌡️ Temperature {timeframe} in {location}: {min_temp}°C to {max_temp}°C. I'd recommend {advice}."

    def generate_activity_advice(self, weather_info, forecast_day, location, timeframe):
        """Generate outdoor activity advice"""
        if forecast_day:
            rain_chance = forecast_day['day'].get('daily_chance_of_rain', 0)
            max_temp = forecast_day['day'].get('maxtemp_c', 20)
        else:
            rain_chance = 20
            max_temp = weather_info.get('temp_c', 20)

        if rain_chance > 50:
            return f"🌧️ Not ideal for outdoor activities {timeframe} in {location}. {rain_chance}% chance of rain."
        elif 15 <= max_temp <= 28 and rain_chance < 30:
            return f"🌞 Perfect weather for outdoor activities {timeframe} in {location}! {max_temp}°C, low rain chance."
        else:
            return f"🌤️ Okay weather for outdoor activities {timeframe} in {location}. Temp: {max_temp}°C, {rain_chance}% rain chance."

    def generate_general_response(self, weather_info, forecast_day, location, timeframe):
        """Generate general weather summary"""
        if forecast_day:
            max_temp = forecast_day['day'].get('maxtemp_c', 20)
            min_temp = forecast_day['day'].get('mintemp_c', 15)
            condition = forecast_day['day'].get('condition', {}).get('text', 'Clear')
        else:
            temp = weather_info.get('temp_c', 20)
            max_temp = min_temp = temp
            condition = weather_info.get('condition', {}).get('text', 'Clear')

        return f"🌤️ Weather {timeframe} in {location}: {condition}, {min_temp}°C to {max_temp}°C."

    def run(self):
        """Main menu loop using pyinputplus"""
        print("🌤️ Welcome to WeatherWise! Your simple weather advisor.")
        print("📝 Note: Using demo weather data for testing purposes.\n")

        while True:
            try:
                choice = pyip.inputMenu(
                    ["Set Location", "Ask a Question", "Exit"],
                    prompt="\nWhat would you like to do?\n",
                    numbered=True
                )

                if choice == "Set Location":
                    self.current_location = pyip.inputStr("Enter your location (e.g., New York, London): ")
                    print(f"✅ Location set to: {self.current_location}")

                elif choice == "Ask a Question":
                    if not self.current_location:
                        print("❌ Please set a location first!")
                        continue

                    print("\n💬 Example questions:")
                    print("• 'Should I take an umbrella tomorrow?'")
                    print("• 'Will it be cold today?'")
                    print("• 'Is it good weather for a picnic?'")

                    question = pyip.inputStr("\nYour weather question: ")

                    print("🔍 Analyzing your question...")
                    parsed = self.parse_weather_question(question)

                    print(f"🌐 Getting weather data for {parsed['location']}...")
                    weather_data = self.get_weather_data(parsed["location"])

                    if weather_data:
                        response = self.generate_weather_response(parsed, weather_data)
                        print(f"\n🎯 WeatherWise Advice:")
                        print("─" * 40)
                        print(response)
                        print("─" * 40)
                    else:
                        print("❌ Failed to fetch weather data. Please try again.")

                elif choice == "Exit":
                    print("👋 Thanks for using WeatherWise! Stay weather-aware!")
                    break

            except KeyboardInterrupt:
                print("\n👋 Thanks for using WeatherWise!")
                break
            except Exception as e:
                print(f"❌ An error occurred: {e}")
                print("Please try again.")

# Instructions for getting a real API key:
def setup_instructions():
    print("""
    🔧 SETUP INSTRUCTIONS:

    1. Get a free API key from https://weatherapi.com
    2. Replace 'YOUR_API_KEY_HERE' in the code with your actual key
    3. For now, the app uses demo data for testing

    📦 Required packages (install with pip):
    - pyinputplus
    - requests
    """)

if __name__ == "__main__":
    setup_instructions()
    app = WeatherWise()
    app.run()

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyinputplus: filename=pyinputplus-0.2.12-py3-none-any.whl size=11348 sha256=6b12faf8289b9c915537c1389c696750cf91a5be03c3b9784cab5f0c367592e0
  Stored in directory: /root/.cache/pip/wheels/a3/6b/89/3bec14288af8e0729d088fb41a785e583d49e772f2529c7676
  Created wheel for pysimplevalidate: filename=pysimplevalidate-0.2.12-py3-none-any.whl size=16225 sha256=9f4bb1af3439fe66b624b223bcc762abec5d75552b846e8a296ab9d95a3d904d
  Stored in directory: /root/.cache/pip/wheels/fc/40/7a/4d3b2dc2e80b4b3bbda89eec94fcefd6c8bd1101cc7bb89554
  Created wheel for stdiomask: filename=stdiomask